<a href="https://colab.research.google.com/github/larajakl/Computational-Linguistics/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://huggingface.co/docs/transformers/main_classes/trainer#transformers.TrainingArguments



In [1]:
!pip install transformers
!pip install datasets
!pip install evaluate
!pip install accelerate --upgrade
!pip install optuna
!pip install optuna-integration[pytorch_lightning]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 12.7 MB/s eta 0:00:00
   

In [2]:
from datasets import load_dataset, DatasetDict
from transformers import DataCollatorWithPadding

from transformers import AutoTokenizer

from transformers import set_seed
from collections import Counter


In [3]:
dataset = load_dataset("mrjunos/depression-reddit-cleaned")

set_seed(24)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

depression-reddit-cleaned.py:   0%|          | 0.00/2.68k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/1.68M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7731 [00:00<?, ? examples/s]

In [4]:
# Check distribution of labels in full dataset:

full_label_distribution = Counter(dataset['train']['label'])
print("Full dataset label distribution:", full_label_distribution)

Full dataset label distribution: Counter({0: 3900, 1: 3831})


In [5]:
# Just take the first n tokens for speed on CPU
def truncate(example):
    return {
        'text': " ".join(example['text'].split()[:100]),
        'label': example['label']
    }

# Random examples for train, validation and test
# Limit the dataset to the first 200 entries, JUST FOR NOW (ADAPT THESE LINES LATER)
subset_dataset = dataset['train'].shuffle(seed=24).select(range(1250))
# Define the train/val/test split proportions:
train_ratio, val_ratio = 0.7, 0.15  # 70% train, 15% val, 15% test
# Shuffle the dataset once:
shuffled_dataset = subset_dataset.shuffle(seed=24)
# Compute the split indices:
total_size = len(shuffled_dataset)
train_end = int(train_ratio * total_size)
val_end = train_end + int(val_ratio * total_size)
# Create splits:
train = shuffled_dataset.select(range(train_end)).map(truncate)
val = shuffled_dataset.select(range(train_end, val_end)).map(truncate)
test = shuffled_dataset.select(range(val_end, total_size)).map(truncate)

# Print the sizes of the splits:
print(f"Train size: {len(train)}, Validation size: {len(val)}, Test size: {len(test)}")

dataset_dict = DatasetDict({
    "train": train,
    "val": val,
    "test": test
})

Map:   0%|          | 0/875 [00:00<?, ? examples/s]

Map:   0%|          | 0/187 [00:00<?, ? examples/s]

Map:   0%|          | 0/188 [00:00<?, ? examples/s]

Train size: 875, Validation size: 187, Test size: 188


In [6]:
print(shuffled_dataset)

print(dataset_dict)

Dataset({
    features: ['text', 'label'],
    num_rows: 1250
})
DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 875
    })
    val: Dataset({
        features: ['text', 'label'],
        num_rows: 187
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 188
    })
})


In [19]:
# Model 1: Distil BERT cased
# cased models: they treat words like "Word" and "word" as separate tokens

tokenizer_distilbert = AutoTokenizer.from_pretrained("distilbert/distilbert-base-cased")

def tokenize_function_distilbert(examples):
    return tokenizer_distilbert(examples["text"], padding=True, truncation=True)

small_tokenized_dataset_distilbert = dataset_dict.map(tokenize_function_distilbert, batched=True, batch_size=16)
data_collator_distilbert = DataCollatorWithPadding(tokenizer=tokenizer_distilbert)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Map:   0%|          | 0/875 [00:00<?, ? examples/s]

Map:   0%|          | 0/187 [00:00<?, ? examples/s]

Map:   0%|          | 0/188 [00:00<?, ? examples/s]

In [ ]:
print(small_tokenized_dataset_distilbert)

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 875
    })
    val: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 187
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 188
    })
})


In [7]:
# Model 2: RoBERTa base
tokenizer_roberta = AutoTokenizer.from_pretrained("roberta-base")

def tokenize_function_roberta(examples):
    return tokenizer_roberta(examples["text"], padding=True, truncation=True)

# Apply the tokenize function to the dataset
small_tokenized_dataset_roberta = dataset_dict.map(tokenize_function_roberta, batched=True, batch_size=16)

# Create a data collator with padding
data_collator_roberta = DataCollatorWithPadding(tokenizer=tokenizer_roberta)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/875 [00:00<?, ? examples/s]

Map:   0%|          | 0/187 [00:00<?, ? examples/s]

Map:   0%|          | 0/188 [00:00<?, ? examples/s]

In [9]:
# Mounting Google Drive to store the checkpoints in Google Drive instead of my runtime:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import numpy as np
import evaluate
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForSequenceClassification

In [25]:
# Training the Distil BERT cased model:

set_seed(24)

model_distilbert = AutoModelForSequenceClassification.from_pretrained('distilbert/distilbert-base-cased', num_labels=2) # 2 labels: depression/no depression
accuracy = evaluate.load("accuracy")
precision = evaluate.load("precision")
recall = evaluate.load("recall")
f1 = evaluate.load("f1")

arguments_distilbert = TrainingArguments(
    output_dir="/content/drive/MyDrive/distilbert1",
    per_device_train_batch_size=16, # adapt
    per_device_eval_batch_size=16, # adapt
    logging_steps=8,
    num_train_epochs=5, # adapt
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5, # adapt
    weight_decay=0.01, # adapt
    load_best_model_at_end=True,
    report_to='none',
    seed=224
)

def compute_metrics(eval_pred):
    """Called at the end of validation."""
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    # Debugging: Print shapes and values
    print(f"Logits shape: {logits.shape}")
    print(f"Labels: {labels}")
    print(f"Predictions: {predictions}")

    # Use scikit-learn for reliable metric computation
    acc = accuracy_score(labels, predictions)
    prec = precision_score(labels, predictions, average="binary", zero_division=0)
    rec = recall_score(labels, predictions, average="binary", zero_division=0)
    f1 = f1_score(labels, predictions, average="binary", zero_division=0)

    return {
        "accuracy": acc,
        "precision": prec,
        "recall": rec,
        "f1": f1
    }


trainer_distilbert = Trainer(
    model=model_distilbert,
    args=arguments_distilbert,
    train_dataset=small_tokenized_dataset_distilbert['train'],
    eval_dataset=small_tokenized_dataset_distilbert['val'], # change to test when you do your final evaluation!
    processing_class=tokenizer_distilbert,
    data_collator=data_collator_distilbert,
    compute_metrics=compute_metrics
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
trainer_distilbert.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.172500,0.169590,0.946524,0.967391,0.927083,0.946809
2,0.113300,0.132969,0.957219,0.968085,0.947917,0.957895
3,0.051000,0.139127,0.951872,0.957895,0.947917,0.952880
4,0.004100,0.141125,0.967914,0.989130,0.947917,0.968085
5,0.040300,0.162952,0.957219,0.968085,0.947917,0.957895


Logits shape: (187, 2)
Labels: [0 0 1 1 0 0 0 0 0 1 1 0 1 1 1 1 1 1 0 1 0 1 1 0 0 0 0 0 0 1 0 0 1 1 1 1 0
 0 1 1 0 0 1 0 0 0 0 1 0 0 1 1 1 1 0 0 0 0 0 0 0 1 1 1 0 0 0 0 1 0 0 0 0 1
 1 1 1 1 0 0 0 0 1 0 1 1 0 0 0 1 0 1 1 1 1 0 1 1 1 0 1 1 1 0 1 0 0 1 0 1 1
 0 0 0 1 0 1 1 0 1 1 1 1 1 1 1 1 0 0 0 0 1 1 0 0 0 0 0 0 1 0 1 0 1 1 0 1 0
 0 1 0 1 0 1 0 1 1 1 1 0 1 0 1 1 0 1 0 1 1 0 1 1 1 1 1 0 1 0 1 0 0 0 1 1 1
 1 1]
Predictions: [0 0 1 1 0 0 0 0 0 1 1 0 1 1 1 1 1 1 0 1 0 1 0 0 0 0 0 0 0 1 0 0 1 1 1 1 0
 0 1 1 0 0 1 0 0 0 0 1 0 0 1 1 1 1 0 0 0 0 0 1 0 1 1 1 0 0 0 0 0 0 0 0 0 1
 1 1 1 1 0 0 0 0 0 0 1 0 0 0 0 1 0 1 1 1 1 0 1 1 1 0 1 1 1 0 1 0 0 1 0 1 1
 0 0 0 1 0 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 1 1 0 0 0 0 0 0 1 0 1 0 0 1 0 1 0
 0 1 0 1 1 1 0 1 1 1 1 0 1 0 1 1 0 1 0 1 0 0 1 1 1 1 1 0 1 0 1 0 0 0 0 1 1
 1 1]
Logits shape: (187, 2)
Labels: [0 0 1 1 0 0 0 0 0 1 1 0 1 1 1 1 1 1 0 1 0 1 1 0 0 0 0 0 0 1 0 0 1 1 1 1 0
 0 1 1 0 0 1 0 0 0 0 1 0 0 1 1 1 1 0 0 0 0 0 0 0 1 1 1 0 0 0 0 1 0 0 0 0 1
 1 1 1 1 0 0 

TrainOutput(global_step=275, training_loss=0.12252040339803154, metrics={'train_runtime': 89.3704, 'train_samples_per_second': 48.954, 'train_steps_per_second': 3.077, 'total_flos': 141128748470076.0, 'train_loss': 0.12252040339803154, 'epoch': 5.0})

In [28]:
# Training the RoBERTa base model:

set_seed(24)

model_roberta = AutoModelForSequenceClassification.from_pretrained('roberta-base', num_labels=2) # 2 labels: depression/no depression
accuracy = evaluate.load("accuracy")
precision = evaluate.load("precision")
recall = evaluate.load("recall")
f1 = evaluate.load("f1")

arguments_roberta = TrainingArguments(
    output_dir="/content/drive/MyDrive/roberta3",
    per_device_train_batch_size=8, # could adapt
    per_device_eval_batch_size=16, # could adapt
    logging_steps=10, # logging is helpful to observe and analyse how the model is performing
    num_train_epochs=5, # could adapt
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5, # could adapt
    weight_decay=0.01, # could adapt
    load_best_model_at_end=True,
    report_to='none',
    seed=224
)

def compute_metrics(eval_pred):
    """Called at the end of validation."""
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    # Debugging: Print shapes and values
    print(f"Logits shape: {logits.shape}")
    print(f"Labels: {labels}")
    print(f"Predictions: {predictions}")

    # Use scikit-learn for reliable metric computation
    acc = accuracy_score(labels, predictions)
    prec = precision_score(labels, predictions, average="binary", zero_division=0)
    rec = recall_score(labels, predictions, average="binary", zero_division=0)
    f1 = f1_score(labels, predictions, average="binary", zero_division=0)

    return {
        "accuracy": acc,
        "precision": prec,
        "recall": rec,
        "f1": f1
    }


trainer_roberta = Trainer(
    model=model_roberta,
    args=arguments_roberta,
    train_dataset=small_tokenized_dataset_roberta['train'],
    eval_dataset=small_tokenized_dataset_roberta['val'], # change to test when you do your final evaluation!
    processing_class=tokenizer_roberta,
    data_collator=data_collator_roberta,
    compute_metrics=compute_metrics
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
# to check the label distribution:
print("Train label distribution:", Counter(train['label']))
print("Validation label distribution:", Counter(val['label']))
print("Test label distribution:", Counter(test['label']))

Train label distribution: Counter({0: 442, 1: 433})
Validation label distribution: Counter({1: 96, 0: 91})
Test label distribution: Counter({1: 98, 0: 90})


In [30]:
trainer_roberta.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.199800,0.211027,0.946524,0.957447,0.937500,0.947368
2,0.092100,0.182098,0.962567,1.000000,0.927083,0.962162
3,0.001800,0.204408,0.967914,0.989130,0.947917,0.968085
4,0.085000,0.192069,0.973262,1.000000,0.947917,0.973262
5,0.057400,0.256893,0.957219,0.958333,0.958333,0.958333


Logits shape: (187, 2)
Labels: [0 0 1 1 0 0 0 0 0 1 1 0 1 1 1 1 1 1 0 1 0 1 1 0 0 0 0 0 0 1 0 0 1 1 1 1 0
 0 1 1 0 0 1 0 0 0 0 1 0 0 1 1 1 1 0 0 0 0 0 0 0 1 1 1 0 0 0 0 1 0 0 0 0 1
 1 1 1 1 0 0 0 0 1 0 1 1 0 0 0 1 0 1 1 1 1 0 1 1 1 0 1 1 1 0 1 0 0 1 0 1 1
 0 0 0 1 0 1 1 0 1 1 1 1 1 1 1 1 0 0 0 0 1 1 0 0 0 0 0 0 1 0 1 0 1 1 0 1 0
 0 1 0 1 0 1 0 1 1 1 1 0 1 0 1 1 0 1 0 1 1 0 1 1 1 1 1 0 1 0 1 0 0 0 1 1 1
 1 1]
Predictions: [0 0 1 1 0 0 0 0 0 1 1 0 1 1 1 1 1 1 1 1 0 1 0 0 0 0 0 0 0 1 0 0 1 1 1 1 0
 0 1 1 1 0 1 0 0 0 0 1 0 0 1 1 1 1 0 0 0 0 0 1 0 1 1 1 0 0 0 0 0 0 0 0 0 1
 1 1 1 1 0 0 0 0 1 0 1 0 0 0 0 1 0 1 0 1 1 0 1 1 1 0 1 1 1 0 1 0 0 1 0 1 1
 0 0 0 1 0 1 1 0 1 1 1 1 1 1 1 1 0 0 0 0 1 1 0 0 0 0 0 0 1 0 1 0 1 1 0 1 0
 0 1 0 1 1 1 0 1 1 1 1 0 1 0 1 1 0 1 0 1 0 0 1 1 1 1 1 0 1 0 1 0 0 0 0 1 1
 1 1]
Logits shape: (187, 2)
Labels: [0 0 1 1 0 0 0 0 0 1 1 0 1 1 1 1 1 1 0 1 0 1 1 0 0 0 0 0 0 1 0 0 1 1 1 1 0
 0 1 1 0 0 1 0 0 0 0 1 0 0 1 1 1 1 0 0 0 0 0 0 0 1 1 1 0 0 0 0 1 0 0 0 0 1
 1 1 1 1 0 0 

TrainOutput(global_step=550, training_loss=0.10835021767511287, metrics={'train_runtime': 175.1718, 'train_samples_per_second': 24.975, 'train_steps_per_second': 3.14, 'total_flos': 257925403681020.0, 'train_loss': 0.10835021767511287, 'epoch': 5.0})

In [31]:
test_results = trainer_roberta.evaluate(small_tokenized_dataset_roberta["test"])
print(test_results)

Logits shape: (188, 2)
Labels: [1 1 0 0 0 1 0 1 0 0 1 1 0 0 0 1 1 0 1 0 1 0 1 1 1 0 1 1 0 0 0 1 0 1 1 0 1
 1 1 1 1 0 1 0 0 0 1 0 0 0 0 1 0 0 1 1 0 1 0 0 0 1 1 1 1 1 0 1 1 0 1 0 0 0
 0 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 1 0 0 1 1 1 1 1 0 1 0 0 0 1 1 1 1 1 1 0
 1 0 0 0 0 1 1 0 1 0 1 0 1 0 1 1 1 0 1 1 1 1 0 0 0 1 1 1 0 1 0 0 0 0 1 0 0
 1 1 1 1 1 1 1 0 0 0 1 1 1 0 1 1 0 0 1 1 0 0 0 1 1 1 1 0 0 1 1 1 1 0 1 0 0
 0 0 0]
Predictions: [1 0 0 0 0 1 0 1 0 0 1 1 0 0 0 1 1 0 1 0 1 0 1 1 1 0 1 0 0 0 0 1 0 1 1 0 1
 1 1 1 1 0 1 0 0 0 1 0 0 0 0 1 0 0 1 1 0 1 0 0 0 1 1 1 1 1 0 1 1 0 1 0 0 0
 0 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 1 0 0 1 0 1 1 1 0 1 0 0 0 1 1 1 1 1 1 0
 1 0 0 0 0 0 1 0 1 0 1 0 1 0 1 1 0 0 1 1 1 1 0 0 0 1 1 1 0 1 0 0 0 0 1 1 0
 1 1 1 1 1 1 1 0 0 0 1 1 1 0 0 1 0 0 0 1 0 0 0 1 1 1 1 0 0 1 1 1 1 0 1 0 0
 0 0 0]
{'eval_loss': 0.21850699186325073, 'eval_accuracy': 0.9574468085106383, 'eval_precision': 0.9891304347826086, 'eval_recall': 0.9285714285714286, 'eval_f1': 0.9578947368421052, 'eval_runtime'

In [27]:
test_results = trainer_distilbert.evaluate(small_tokenized_dataset_distilbert["test"])
print(test_results)

Logits shape: (188, 2)
Labels: [1 1 0 0 0 1 0 1 0 0 1 1 0 0 0 1 1 0 1 0 1 0 1 1 1 0 1 1 0 0 0 1 0 1 1 0 1
 1 1 1 1 0 1 0 0 0 1 0 0 0 0 1 0 0 1 1 0 1 0 0 0 1 1 1 1 1 0 1 1 0 1 0 0 0
 0 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 1 0 0 1 1 1 1 1 0 1 0 0 0 1 1 1 1 1 1 0
 1 0 0 0 0 1 1 0 1 0 1 0 1 0 1 1 1 0 1 1 1 1 0 0 0 1 1 1 0 1 0 0 0 0 1 0 0
 1 1 1 1 1 1 1 0 0 0 1 1 1 0 1 1 0 0 1 1 0 0 0 1 1 1 1 0 0 1 1 1 1 0 1 0 0
 0 0 0]
Predictions: [1 0 0 0 0 1 0 1 0 0 1 1 0 0 0 1 1 0 1 0 1 0 1 1 1 0 1 1 0 0 0 1 0 1 1 0 1
 1 1 1 1 0 1 0 0 0 1 0 0 0 0 1 0 0 1 1 0 1 0 0 0 1 1 1 0 1 0 1 1 0 1 0 0 0
 0 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 1 0 0 1 1 1 1 1 0 1 0 0 0 1 1 1 1 1 1 0
 1 0 0 0 0 0 1 0 1 0 1 0 1 0 1 1 1 0 1 1 1 1 0 0 0 1 1 1 0 1 0 0 0 0 1 0 0
 1 1 1 1 1 1 1 0 0 0 1 1 1 0 0 1 0 0 0 1 0 0 0 1 1 1 1 0 0 1 1 1 1 0 1 0 0
 0 0 0]
{'eval_loss': 0.0843702107667923, 'eval_accuracy': 0.973404255319149, 'eval_precision': 1.0, 'eval_recall': 0.9489795918367347, 'eval_f1': 0.9738219895287958, 'eval_runtime': 0.6646, 'eval_s

In [ ]:
# testing in initial round:
predictions = trainer_roberta.predict(small_tokenized_dataset_roberta['test'])
predicted_labels = np.argmax(predictions.predictions, axis=-1)
true_labels = predictions.label_ids

for i, (true, pred) in enumerate(zip(true_labels, predicted_labels)):
    if true != pred:
        # Get the corresponding text for the misclassified sample
        misclassified_text = small_tokenized_dataset_roberta['test'][i]['text']
        print(f"Sample {i}:")
        print(f"True label = {true}, Predicted label = {pred}")
        print(f"Text: {misclassified_text}")
        print("-" * 50)

Sample 0:
True label = 1, Predicted label = 0
Text: what do you guy think will this finally change my life buspasfar mg day escitalopram 0mg bupropion 0
--------------------------------------------------
Sample 13:
True label = 1, Predicted label = 0
Text: mai asher 9 lynnestactia the guy look depressed depression is real
--------------------------------------------------


In [ ]:
# In the following code cells, I use Optuna to test hyperparameters. New needed imports:
import optuna
import torch

In [ ]:
# Optuna hyperparameter tuning:

In [ ]:
# Testing:

metric = evaluate.load("accuracy")
fine_tuned_model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/XXXXXXXXXXXXX")

model_inputs = tokenizer(small_tokenized_dataset['test']['text'], padding=True, truncation=True, return_tensors='pt')
outputs = fine_tuned_model(**model_inputs, output_hidden_states=True)

predictions = torch.argmax(outputs.logits, dim=-1)
accuracy = metric.compute(predictions=predictions, references=small_tokenized_dataset['test']['label'])
print(accuracy)

In [ ]:
# Visualisations:

!pip install bertviz transformers
!pip install bertviz

In [ ]:
import os
from torch.utils.tensorboard import SummaryWriter
import re
import tensorflow as tf
import tensorboard as tb  #this is important to be able to store embeddings in a format so that I can use tensorflow visualiation

In [ ]:
# Minimum 2 visualisations -> of ONE model!
# (Every change in hyperparameters is a new model.)
# Make 2 visualisations of the FINAL model only (in 2 checkpoints/layers)
# For your project, you need to use a separate test set for the visualisations.

In [ ]:
# ADAPT FOR MY OWN DATASET: (look at tutorial 5 for further steps and details)

# the code associates our evaluation step with movie review and label

# for project: load different layers from different epochs to see how it changes!

path = "results_vis" # creates directory, can change it to drive location
layer=0 # sets layer i want to start from
if not os.path.exists(path):
  os.mkdir(path)

while layer in range(len(outputs['hidden_states'])):
  if not os.path.exists(path+'/layer_' + str(layer)):
    os.mkdir(path+'/layer_' + str(layer))

  example = 0
  tensors = []
  labels = []

  while example in range(len(outputs['hidden_states'][layer])):
    sp_token_position = 0
    for token in model_inputs['input_ids'][example]:
      if token != 101:
        sp_token_position += 1
      else:
        tensor = outputs['hidden_states'][layer][example][sp_token_position]
        tensors.append(tensor)
        break

    label = [small_tokenized_dataset['val']['text'][example],str(small_tokenized_dataset['val']['label'][example])]
    labels.append(label)
    example +=1

  writer=SummaryWriter(path+'/layer_' + str(layer))
  writer.add_embedding(torch.stack(tensors), metadata=labels, metadata_header=['Text','Emotion'])

  layer+=1